# Signal extraction from CUBIC 3D image data using ilastik analysis

This tutorial provides instructions for how to extract the signals from CUBIC 3d image data using ilastik. [The ilastik (interactive learning and segmentation toolkit)](https://www.ilastik.org/) is a Python library of interactive machine learning for (bio)image analysis. It implements leverage machine learning algorithms to easily segment, classify, track and count your cells or other experimental data developed by Berg et al., Nat Methods (2019). **Note. Original dataset has 454 images, but it's too large to work on Google Colab, therefore we picked up from 201th to 210th images to reduce the data size. h5 file is created after machine learning by ilastik software.**

## Import required libraries

In [ ]:
import tifffile
import glob
import h5py
import os
import numpy as np
import string
from scipy.ndimage import label
from matplotlib import pyplot as plt
import pandas as pd
import scipy.ndimage as ndi
import glob

## Define required functions

A function to read tiff files

In [ ]:
def load_tiff_sequence ( imdir, imgtype='tiff', range=None ):
    """
    load tiff sequence stored in the same directory
    e.g. 
    vol = load_tiff_sequence (imgdir, '.png', range=[])
    """

    imlist = glob.glob( imdir + '*.' + imgtype )
    imlist.sort() # sort numerically
    
    if range is not None:
        imlist = imlist[ range[0]:range[1]]
        
    #get image properties by reading the first image
    im = tifffile.imread(imlist[0])
    imsize_x = im.shape[1]
    imsize_y = im.shape[0]
    imsize_z = len( imlist )
    imsize = ( imsize_z, imsize_y, imsize_x )
    imtype = im.dtype
    
    stack = np.zeros( imsize, dtype=imtype )
    for (i, impath) in enumerate(imlist):
        im = tifffile.imread( impath )
        stack[i,:,:] = im
        
    return stack

A function to export as hdf5 file

In [ ]:
def write_as_hdf5( stack, h5name, destname, 
                   chunks_enabled=True, chunksize=None,
                   attributes=None ):
    """
    e.g.
    write_as_hdf5(vol, 'test.hdf5', 'resolution_0', True, (100,100,100))
    """
    if chunks_enabled:
        if chunksize is None:
            chunks = True
        else:
            chunks = chunksize
    else:
        chunks = None
        
    with h5py.File( h5name, 'w', driver='stdio' ) as hf:
        data = hf.create_dataset (destname,
                                  chunks=chunks,
                                  data=stack )
        if attributes is not None:
            for key, value in attributes.items():
                data.attrs[key] = value

A function to return the size of a hdf5 file

In [ ]:
def ask_hdf5_size( h5name, dsetname=None ):
    
    # obtain file handle
    hf = h5py.File( h5name, 'r' )
    
    if dsetname is None:
        # get the name of the 0th dataset
        dsetname = list( hf.keys() )[0]
        dset = hf[ dsetname ]
    else:
        # get dataset
        dset = hf[ dsetname ]
    
    # print size
    print( "Data set size:", dset.shape )
    
    # close handle
    hf.close()

A function to read hdf5 files

In [ ]:
def load_hdf5( h5name, dsetname=None, multichannel=True ):
    
    # obtain file handle
    hf = h5py.File( h5name, 'r' )
    
    if dsetname is None:
        # get the name of the 0th dataset
        dsetname = list( hf.keys() )[0]
        dset = hf[ dsetname ]
    else:
        # get dataset
        dset = hf[ dsetname ]
    
    if multichannel:
        # load data as numpy array
        data = dset[ :, :, :, 0] # 0th channel = cells
        #data = dset[ :, :, :, 0] # 0th channel = cells
    else:
        data = dset[ :, :, :] # 0th channel = cells
        #data = dset[ :, :, :] # 0th channel = cells

    # close handle
    hf.close()
    
    return data

A function to binarize probability images by thresholding

In [ ]:
def calculate_prob_hdf5(file_list, threshold):
    
    # load probabiltiy image
    prob = load_hdf5( file, "expmat", multichannel=True )
    print (prob.shape)
    
    ### Binarize probability image
    thresh = threshold * 255
    binary = ( prob > thresh )
    print ("Total volume of detected signals:", binary.sum()*8.25*8.25*10)
    
    # this defines "connectivity" between voxels
    # structure = ndi.generate_binary_structure( 3, 3 )
    
    # this defines "connectivity" between voxels
    structure = np.array( [[[0,0,0],
                            [0,0,0],
                           [0,0,0]],
                           [[0,0,0],
                            [0,0,0],
                            [0,0,0]],
                           [[0,0,0],
                            [0,0,0],
                            [0,0,0]]])
        
    # Label isolated objects
    objects, num_objects = label( binary, structure )
    print( "Number of detected objects:", objects.max() )
        
    # make binary into uint16
    binary16 = (255*binary).astype( 'uint16' )
    
    # export as tiff
    basename = os.path.basename(file)
    tiffdir = rootdir + "tiff"
    if not os.path.exists(tiffdir):
      os.mkdir(tiffdir)
    filename = rootdir + "tiff/" + basename[:-26] + f"_p{int(threshold*100)}_all_639.tiff"
    tifffile.imsave( filename, binary16 )
    
    ### Find center of mass
    ids = np.arange( 1, num_objects+1 )
    coms = ndi.center_of_mass( binary, objects, ids )
    
    # convert to numpy array
    coms = np.array( coms )
    
    # Compute volume of each object
    unique, counts = np.unique( objects, return_counts=True )
    # remove 0
    unique = unique[1:]
    counts = counts[1:]
    
    # create empty dataframe
    df = pd.DataFrame()
    
    # colum "ID"
    df['ID'] = unique
    
    # column "X", "Y", "Z"
    df['X'] = coms[ :, 2 ]
    df['Y'] = coms[ :, 1 ]
    df['Z'] = coms[ :, 0 ]
    
    # colum "volume"
    df["volume"] = counts
    
    # save as csv
    basename = os.path.basename(file)
    csvdir = rootdir + "csv"
    if not os.path.exists(csvdir):
      os.mkdir(csvdir)
    filename = rootdir + "csv/" + basename[:-26] + f"_p{int(threshold*100)}_all_639.csv"
    df.to_csv( filename, index=False, float_format='%.2f' )

## Data Download

In [ ]:
!wget https://www.dropbox.com/s/z0cmicvosckuqqr/190604_%23144_lung_raw_tiff_10slices.zip
!wget https://www.dropbox.com/s/al85vb3bxwl250g/190604_P_%23144_lung_ctrl_x125_639_Probabilities_10slices.h5

! unzip 190604_#144_lung_raw_tiff_10slices.zip

--2022-08-12 01:08:18--  https://www.dropbox.com/s/z0cmicvosckuqqr/190604_%23144_lung_raw_tiff_10slices.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/z0cmicvosckuqqr/190604_%23144_lung_raw_tiff_10slices.zip [following]
--2022-08-12 01:08:19--  https://www.dropbox.com/s/raw/z0cmicvosckuqqr/190604_%23144_lung_raw_tiff_10slices.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5263c131ee2dab783539ded818.dl.dropboxusercontent.com/cd/0/inline/Bq39375uK6TNyJt1cXp7U0RKPCJecdKsVl4_byHSDdT9IPYFnV1c32UjyHeApzYMEqZzEukShP2EFChE2HKCfY0f9-nnpcUlA_6iCRr-jURIugeEo0f5Dg0cxIzxR2ZzZ2IYgxQtG5ZwXLJ2WbfmEnBCoqeaj8OtORr1plK-hAwLTQ/file# [following]
--2022-08-12 01:08:19--  https://uc5263c131ee2dab783539ded818.dl.dropboxusercontent.

Choose Tiff file folder

In [ ]:
os.chdir("/content/190604_#144_lung_raw_tiff_10slices")
print(os.getcwd())

/content/190604_#144_lung_raw_tiff_10slices


Read Tiff file

In [ ]:
imgdir = "/content/190604_#144_lung_raw_tiff_10slices/"
img = load_tiff_sequence( imgdir, imgtype='tiff')

print(img.shape)

(10, 2160, 2560)


Save as hdf5

In [ ]:
filename = "/content/190604_P_#144_lung_ctrl_x125_639_10slices.hdf5"
dname = "content"

write_as_hdf5( img, filename, dname, chunks_enabled=True, chunksize=(10,100,100) )

## Binarization of probability images by thresholding

In [ ]:
h5name = "/content/190604_P_#144_lung_ctrl_x125_639_Probabilities_10slices.h5"
hf = h5py.File( h5name, "r" )

In [ ]:
data = hf["expmat"]
print (data.shape)
l1_prob = data[:,:,:,0] # probability of label1

(10, 2160, 2560, 2)


In [ ]:
thresholds = [24, 50, 75, 101, 126, 152, 178, 203, 229]
for thresh, percent, idx in zip(thresholds, range(10, 100, 10), list(string.ascii_uppercase)):
  # maks a binary mask
  binary = (l1_prob > thresh)
  print (binary.sum()*8.25*8.25*10)
  # make binary into uint8
  binary = (255*binary).astype('uint16')

165508942.5
133056061.875
119030422.5
103172540.625
85788016.875
72314364.375
59966465.625
49488243.75
33390781.875


Define root diretory

In [ ]:
rootdir = "/content/"

Get files which ends with 'probability'

In [ ]:
file_list = glob.glob( rootdir + "*_Probabilities_10slices.h5" )
print( file_list )

['/content/190604_P_#144_lung_ctrl_x125_639_Probabilities_10slices.h5']


In [ ]:
file = file_list[0]
prob = load_hdf5( file, "expmat", multichannel=False )

Loop through all files and thresholds

In [ ]:
thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]
for file in file_list:
    for thresh in thresholds:
      ask_hdf5_size( file, dsetname=None )
      raw = load_hdf5(file, multichannel=False)
      print (file.rsplit("/")[-1][:-3])
      calculate_prob_hdf5(file, thresh)

Data set size: (10, 2160, 2560, 2)
190604_P_#144_lung_ctrl_x125_639_Probabilities_10slices
(10, 2160, 2560)
Total volume of detected signals: 160266768.75
Number of detected objects: 235470
Data set size: (10, 2160, 2560, 2)
190604_P_#144_lung_ctrl_x125_639_Probabilities_10slices
(10, 2160, 2560)
Total volume of detected signals: 117569120.625
Number of detected objects: 172737
Data set size: (10, 2160, 2560, 2)
190604_P_#144_lung_ctrl_x125_639_Probabilities_10slices
(10, 2160, 2560)
Total volume of detected signals: 84283835.625
Number of detected objects: 123833
Data set size: (10, 2160, 2560, 2)
190604_P_#144_lung_ctrl_x125_639_Probabilities_10slices
(10, 2160, 2560)
Total volume of detected signals: 59966465.625
Number of detected objects: 88105
Data set size: (10, 2160, 2560, 2)
190604_P_#144_lung_ctrl_x125_639_Probabilities_10slices
(10, 2160, 2560)
Total volume of detected signals: 33390781.875
Number of detected objects: 49059
